## Programme permettant de simuler un gaz parfait 2D avec des particules dans une boite

On distribue $N$ particules en leur donnant soit des vitesses aléatoires ou régulières et des positions aléatoires ou régulières.

On peut changer leur rayon. 

On considère des particules identiques, sous forme de disque ayant des interactions élastiques entre elles ou avec la parroi. 

Il est intéressant de voir l'évolution de la distribution des vitesses avec le temps quand on donne au départ la même vitesse à toutes les particules (thermalisation).

Faire un film intéressant (de longueur suffisante) demande vite 500 pas ou plus et prend un peu de temps.


In [ ]:
import os, math, pylab
import numpy as np
import numpy.random
import io
import base64
from IPython.display import HTML

In [ ]:
def wall_time(pos_a, vel_a, sigma):
    """Fonction qui détermine le prochain choc d'une particule avec un mur."""
    if vel_a > 0.0:
        del_t = (1.0 - sigma - pos_a) / vel_a
    elif vel_a < 0.0:
        del_t = (pos_a - sigma) / abs(vel_a)
    else:
        del_t = float('inf')
    return del_t

In [ ]:
def pair_time(pos_a, vel_a, pos_b, vel_b, sigma):
    """Fonction qui détermine le temps du prochain choc d'une particule avec une autre."""
    del_x = [pos_b[0] - pos_a[0], pos_b[1] - pos_a[1]]
    del_x_sq = del_x[0] ** 2 + del_x[1] ** 2
    del_v = [vel_b[0] - vel_a[0], vel_b[1] - vel_a[1]]
    del_v_sq = del_v[0] ** 2 + del_v[1] ** 2
    scal = del_v[0] * del_x[0] + del_v[1] * del_x[1]
    Upsilon = scal ** 2 - del_v_sq * (del_x_sq - 4.0 * sigma ** 2)
    if Upsilon > 0.0 and scal < 0.0:
        del_t = - (scal + math.sqrt(Upsilon)) / del_v_sq
    else:
        del_t = float('inf')
    return del_t

In [ ]:
def min_arg(l):
    """Récupère à la fois le minimum d'une liste et l'indice correspondant à ce minimum."""
    return min(zip(l, range(len(l))))


In [ ]:
def compute_next_event(pos, vel):
    """ Détermination du prochain "évènement", c'est-à-dire l'instant de ce 
    choc et la particule (ou la paire) correspondante. À noter que l'on stocke 
    toutes ces infos dans un seul indice (cf disjonction de cas dans 
    compute_new_velocities)."""
    wall_times = [wall_time(pos[k][l], vel[k][l], sigma) for k, l in singles]
    pair_times = [pair_time(pos[k], vel[k], pos[l], vel[l], sigma) for k, l in pairs]
    return min_arg(wall_times + pair_times)


In [ ]:
def compute_new_velocities(pos, vel, next_event_arg):
    """Calcul des nouvelles vitesses"""
    if next_event_arg < len(singles): # Cas d'un choc avec le mur
        collision_disk, direction = singles[next_event_arg]
        vel[collision_disk][direction] *= -1.0 # seule la vitesse sur cet axe est modifiée
    else:                             # Cas d'un choc entre deux particules de même masse
        a, b = pairs[next_event_arg - len(singles)]
        del_x = [pos[b][0] - pos[a][0], pos[b][1] - pos[a][1]]
        abs_x = math.sqrt(del_x[0] ** 2 + del_x[1] ** 2)
        e_perp = [c / abs_x for c in del_x]
        del_v = [vel[b][0] - vel[a][0], vel[b][1] - vel[a][1]]
        scal = del_v[0] * e_perp[0] + del_v[1] * e_perp[1]
        for k in range(2):
            vel[a][k] += e_perp[k] * scal
            vel[b][k] -= e_perp[k] * scal


In [ ]:
def snapshot(t, pos, vel, colors, X, Y, arrow_scale=.2):
    """ La routine qui s'occupe des tracés graphiques."""
    global img
    nbmax = 20
    pylab.subplots_adjust(left=0.0, right=1, top=1, bottom=0)
    pylab.gcf().set_size_inches(12, 12*2/3)
    # Le premier sous-plot: carré de 2x2
    ax1 = pylab.subplot2grid((2,3),(0,0),colspan=2,rowspan=2)
    pylab.setp(pylab.gca(), xticks=[0, 1], yticks=[0, 1])
    pylab.plot(X,Y,'k',lw=0.8) # On y met le trajet de la dernière particule
    pylab.xlim((0,1))   # On doit astreindre les côtés horizontaux
    pylab.ylim((0,1))   # et verticaux
    # Boucle sur les points pour rajouter les cercles colorés
    for (x, y), c in zip(pos, colors): 
        circle = pylab.Circle((x, y), radius=sigma, fc=c)
        pylab.gca().add_patch(circle)
    dx,dy = vel[-1] * arrow_scale # La dernière particule a droit à son vecteur vitesse
    pylab.arrow( x, y, dx, dy, fc="r", ec="r", head_width=0.02, head_length=0.05 )
    pylab.text(.5, 1.03, 't = %.2f' % t, ha='center')
    # Second sous-plot: histogramme de la projection suivant x et y des vitesses
    ax2 = pylab.subplot2grid((2,3),(0,2),colspan=1,rowspan=1)
    r = (-2,2)  # Intervalle de vitesses regardé
    pylab.hist(vel[:,0],bins=20,range=r,alpha=0.5)
    pylab.hist(vel[:,1],bins=20,range=r, alpha=0.5)
    pylab.xlim(r)
    pylab.ylim((0,nbmax))
    pylab.ylabel("Nombre de particules")
    pylab.xlabel("$v_x$ (bleu), $v_y$ (orange)")
    # Troisième sous-plot: histogramme de la norme des vitesses
    ax3 = pylab.subplot2grid((2,3),(1,2),colspan=1,rowspan=1)
    r = (0,2)   # Intervalle de vitesses regardé
    v = np.linspace(0,2,100)
    mb = 10*maxbol(v,kbt)
    pylab.plot(v,mb)
    pylab.hist(np.sqrt(np.sum(vel**2,axis=1)),bins=20,range=r)
    pylab.plot(v)
    pylab.xlim(r)
    pylab.ylim((0,nbmax))
    pylab.ylabel("Nombre de particules")
    pylab.xlabel("$||\\vec{v}||$")
    pylab.savefig(os.path.join(output_dir, '{:04d}.png'.format(img)), bbox_inches="tight")
    img += 1


In [ ]:
def check_position():
    """ Une routine pour s'assurer que les particules ne se chevauchent pas au 
    départ. Il peut se passer un certain temps avant que l'on trouve une 
    configuration adéquate. """
    continue_condition = True  # Condition de non-arrêt
    c = 0                      # Compteur
    d2= 4*sigma**2             # Distance (carrée) de sécurité
    while continue_condition:
        c += 1
        if c%100 == 0:         # Un peu de feedback
           print(c,'trials to get initial conditions and still trying...')
        pos = np.random.random((N,2))*(1-2*sigma) + sigma
        k = 0
        for (i,j) in pairs:    # Les vérifications sur toutes les paires
            if sum((pos[i]-pos[j])**2) > d2: k+= 1
            else:
                if c%100 == 0: print(i,j)
                break
        if k == len(pairs): continue_condition = False
    print("Let's compute some physics !")
    return pos


Maxwell-Bolzmann a 2D

In [ ]:
def maxbol(v,kbt):
    mb=(v/kbt)*np.exp(-0.5*v**2/kbt)
    return(mb)
    

In [ ]:
def reg_position(N):
    dist_ini= 1 / (np.sqrt(N)+1)
    pos2=np.linspace(dist_ini,1-dist_ini,int(np.sqrt(N)))
    pos3=[]
    for i in pos2:
        for j in pos2:
            pos3.append([i,j])
    pos=pos3
    return(pos)

In [ ]:
def reg_vel(N):
    vel = np.zeros((N,2))+(0.5,0) # tout le monde part vers la droite avec vx=0.5 
    return(vel)

## Les paramètre à changer

Nombre de particules, nombre de chocs, est-ce que la distribution des particules est régulères ou aléatoires, est-ce que les particules ont une distribution de vitesse aléatoires ou elles ont toutes la même vitesse?

On peut aussi essayer "refaire" le film inverse en partant de la dernière valeur des positions et vitesses et en inversant les vitesses. On remarque qu'il ne faut pas beaucoup de particules pour que ça ne marche pas. 

Penser à changer e nom du film pour ne pas écraser l'ancien...

In [ ]:
N = 100
vel_save = np.random.random((N,2))*2 - 1

In [ ]:
# options for main program
reg_pos= 0 # 0: random initial positions; 1 regular initial positions
id_vel = 1  # 0: random initial velocities; 1: all initial velocities equal
reverse = 0 # 0: restart; 1: start with stored (random) velocities 2: start with end positions of last run and inversed velocities

n_steps = 500 # Nombre de chocs

moviename='Film_test.mp4'



In [ ]:
#  main program

output_dir = "PNG/T1_particules_en_boite_libre_movie"

cmd3 = 'rm ' + output_dir + '/*.png'
os.system(cmd3)                    #On efface les anciennes images

colors = ['grey']*N
colors[-1]='red'

img = 0
if not os.path.exists(output_dir): os.makedirs(output_dir)
    
sigma = 0.01                       # Rayon des particules
singles = [(i,j) for i in range(N) for j in range(2)]   # L'ensemble des particules (en 2D)
pairs = [(i,j) for i in range(N) for j in range(i+1,N)] # L'ensemble des paires

if reverse == 2:
    vel = -vel
    pos = pos # well, OK
    print('if1')
elif reverse == 1:
    vel = vel_save.copy() # This SHOULD take the original velocities from vel_save
    # BUT for whatever reason, sometimes, vel_save *IS* (slightly) altered, so that one cannot 
    # restart a calculation with the same conditions ???
    pos = reg_position(N)
    print('if2')
elif reverse == 0 :
    print('if3')
    if reg_pos == 0:
        print('if4')
        pos = check_position()             # Sélection des positions (de manière aléatoire)
    else :                                 # disposition régulière
        print('if5')
        pos = reg_position(N)  
    if id_vel == 1 :
        print('if6')
        vel = np.zeros((N,2))+(0.5,0) # tout le monde part vers la droite avec vx=0.5 
    else : vel = np.random.random((N,2))*2 - 1 # selection aléatoire des vitesses initiales

#print('vel_save avant')        
#print(vel_save)

#print('vel initial')
#print(vel)

X,Y = [pos[-1][0]],[pos[-1][1]]    # La dernière particule va être suivie à la loupe

t = 0.0                            # Temps initial
dt = 0.01                          # interval 


vmoysq=0
for i in range(len(vel)):
    vmoysq += vel[i][0]**2+vel[i][1]**2

vmoysq=vmoysq/len(vel) # vitesse quadratique moyenne au carré
kbt=0.5*vmoysq # parceque m=1 donne la température du gaz
print(kbt)


next_event, next_event_arg = compute_next_event(pos, vel) # On calcule la première étape

print(next_event)
print(next_event_arg)

snapshot(t, pos, vel, colors, X, Y)# et on prend une première photo.
for step in range(n_steps): 
    tzero = t
    while t <= tzero + next_event : # Début des calculs jusqu'à la prochaine sortie
        t += dt            # On avance
        pos += vel * dt    # On met à jour les position
        X.append(pos[-1][0])           # Suivi x de la dernière particule
        Y.append(pos[-1][1])           # Ainsi que Y
        snapshot(t,pos,vel,colors,X,Y) # Souriez pour la photo
        print(t)
    compute_new_velocities(pos, vel, next_event_arg)
    next_event, next_event_arg = compute_next_event(pos, vel)
    print('pas' , step, 'sur', n_steps)                # et un peu de feedback



In [ ]:
cmd1 = 'rm ' + moviename
#print(cmd1)
os.system(cmd1) 

#filter_complex 'scale=864:576, pad=720:576:71:57'

cmd2 = 'ffmpeg -framerate 24 -pattern_type glob -i \'' + output_dir + '/*.png\'  -vcodec libx264 -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -pix_fmt yuv420p ' + moviename

print("Execution de la commande de conversion")
print(cmd2)
os.system(cmd2)
print("Fin de la commande de conversion")


In [ ]:
import io
import base64
from IPython.display import HTML

In [ ]:
video = io.open(moviename, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

Notebook by Cécile Hébert (2019-2023).
Except where otherwise noted, the content of this notebook is licensed under MIT licence.